In [3]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise import Dataset
from surprise import Reader
from surprise.prediction_algorithms.knns import KNNWithZScore, KNNBaseline
from surprise.prediction_algorithms.matrix_factorization import NMF
from pandas.io.json import json_normalize
from pymongo import MongoClient

In [4]:
def convert_ids(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('int64')

In [39]:
ratings_df = pd.read_csv('../data/the-movies-dataset/ratings_small.csv')
movies_df = pd.read_csv('../data/the-movies-dataset/movies_metadata.csv'
                        , converters={'id': lambda x: convert_ids(x), 'imdb_id': lambda x: convert_ids(x)}
                       ,usecols=['id', 'original_title', 'belongs_to_collection'
                                 , 'budget', 'genres', 'homepage'
                                 ,'imdb_id', 'overview', 'popularity', 'poster_path'
                                 , 'production_companies','release_date', 'revenue', 'runtime',
                                 'spoken_languages', 'status', 'tagline', 'title', 'video',
                                 'vote_average', 'vote_count'])

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
# my_favorite_movies = {7345: 5, #TwBB
# #                     4995: 4.5, #Boogie Nighs
# #                     11017:3.5, #Billy Madison
# #                     805: 4,#Rosemnarys baby 
# #                     9429: 2.5,#Roxbury
# #                     8872: 4} #Wayne's World
# rows = []
# for k, v in my_favorite_movies.items():
#     rows.append({'userId': 9999999
#                 ,'movieId': k
#                 ,'rating': v})
#                 #,'timestamp': pd.Timestamp(pd.datetime.now(), unit='s').timestamp()})
# #user -- 99999
# #movies
# #ratings_df.describe()
# rows
# new_tail = pd.DataFrame(rows, index=(['Blake1', 'Blake2', 'Blake3', 'Blake4', 'Blake5', 'Blake6', ]))


ratings_df = ratings_df.append(new_tail)

In [52]:
###May need Fuzzy matching, but for now:
movies_df = movies_df[movies_df.spoken_languages == """[{'iso_639_1': 'en', 'name': 'English'}]"""]


ratings_with_movie_names = ratings_df.merge(movies_df[['id', 'original_title']], how='left', left_on='movieId', right_on='id')
ratings_with_movie_names = ratings_with_movie_names[ratings_with_movie_names.original_title.isnull() == False]

In [53]:
algo = SVD(verbose=True)
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings_with_movie_names[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()

In [54]:
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [55]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8919  0.8933  0.9064  0.8936  0.9099  0.8990  0.0075  
MAE (testset)     0.6878  0.6916  0.7055  0.6852  0.6987  0.6937  0.0074  
Fit time          0.90    0.86    0.86    0.80    0.79    0.84    0.04    
Test time         0.02    0.02    0.02    0.02    0.03    0.02    0.00    


{'test_rmse': array([0.89191682, 0.89328177, 0.90639806, 0.89363411, 0.90985345]),
 'test_mae': array([0.68776186, 0.69163342, 0.7054838 , 0.68517148, 0.69867408]),
 'fit_time': (0.8998048305511475,
  0.8597760200500488,
  0.8620948791503906,
  0.7976369857788086,
  0.7860798835754395),
 'test_time': (0.024932146072387695,
  0.02443408966064453,
  0.02268695831298828,
  0.02268505096435547,
  0.02855706214904785)}

In [56]:
from collections import defaultdict
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


In [57]:
testset = trainset.build_anti_testset()

In [58]:
predictions = algo.test(testset)

In [72]:
top_n = get_top_n(predictions, 100)

In [65]:
predicted_movies_by_name = defaultdict(list)

for key, value in top_n.items():
    predicted_movies_by_name[key] = [get_movie_name(mov_id[0]) for mov_id in value]

In [74]:
my_n = top_n[9999999]
my_n

[(318, 4.736636264334891),
 (923, 4.640634145156084),
 (2019, 4.547498124539418),
 (1945, 4.539636723107582),
 (926, 4.5083727169181484),
 (898, 4.498932642798354),
 (260, 4.49240167638287),
 (475, 4.456741441308257),
 (858, 4.452835144600857),
 (903, 4.451256995838255),
 (2064, 4.389024562845169),
 (3019, 4.347043193225964),
 (8665, 4.344694951675203),
 (3469, 4.341325476295896),
 (3035, 4.339376476879017),
 (1252, 4.338702057953397),
 (4226, 4.334743301256725),
 (2324, 4.3288604765083125),
 (968, 4.321297376106129),
 (49530, 4.321166145677323),
 (994, 4.320956191091358),
 (232, 4.318430714468692),
 (928, 4.3092546931154185),
 (44191, 4.302508372857142),
 (58559, 4.301450635576745),
 (3030, 4.2980194151557045),
 (48780, 4.2876305566784865),
 (3088, 4.284419949202089),
 (6016, 4.2815581144813395),
 (89492, 4.280810785789692),
 (671, 4.26388949116559),
 (2186, 4.261495991516019),
 (233, 4.253701124121904),
 (501, 4.2502358387850245),
 (2020, 4.249767733172723),
 (1262, 4.241089726535762

In [30]:
ratings_with_movie_names.head()

,userId,movieId,rating,timestamp,id,original_title
10,1,1371,2.5,1260759135,1371.0,Rocky III
11,1,1405,1.0,1260759203,1405.0,Greed
13,1,2105,4.0,1260759139,2105.0,American Pie
15,1,2193,2.0,1260759198,2193.0,My Tutor
16,1,2294,2.0,1260759108,2294.0,Jay and Silent Bob Strike Back


In [43]:
ratings_with_movie_names[ratings_with_movie_names.id == my_n[1][0]]['original_title'].iloc[0]

'The Thomas Crown Affair'

In [61]:
def get_movie_name(movie_id):
    return ratings_with_movie_names[ratings_with_movie_names.id == movie_id]['original_title'].iloc[0]

In [49]:
get_movie_name(858)

'Sleepless in Seattle'

In [75]:
[get_movie_name(mov_id[0]) for mov_id in my_n]

['The Million Dollar Hotel',
 'Dawn of the Dead',
 'Hard Target',
 'Nell',
 'Galaxy Quest',
 'Birdman of Alcatraz',
 'The 39 Steps',
 'Bonnie and Clyde',
 'Sleepless in Seattle',
 'Cool Hand Luke',
 'While You Were Sleeping',
 'Dr. Jekyll and Mr. Hyde',
 'K-19: The Widowmaker',
 'Far from the Madding Crowd',
 'Frankenstein',
 'Lonely Hearts',
 'Shriek If You Know What I Did Last Friday the Thirteenth',
 'Local Color',
 'Dog Day Afternoon',
 'In Time',
 'Straw Dogs',
 'Rumble Fish',
 'Gremlins 2: The New Batch',
 'Loose Screws',
 'Confession of a Child of the Century',
 'End of the World',
 'Boat',
 'My Darling Clementine',
 'The Good Thief',
 'This Is 40',
 "Harry Potter and the Philosopher's Stone",
 'Within the Woods',
 'The Wanderers',
 'Grizzly Man',
 'The Bachelor',
 'Stranger Than Fiction',
 'Murder She Said',
 'Dr. Jekyll and Mr. Hyde',
 'Meet the Robinsons',
 'Space Jam',
 'Flags of Our Fathers',
 'Beetlejuice',
 'Laura',
 'Point Break',
 'Broken Flowers',
 'Totally Blonde',
 '

In [41]:
ratings_with_movie_names[ratings_with_movie_names.id == 858]

,userId,movieId,rating,timestamp,id,original_title
180,4,858,5.0,949779022,858.0,Sleepless in Seattle
370,5,858,2.5,1163373651,858.0,Sleepless in Seattle
602,8,858,5.0,1154400181,858.0,Sleepless in Seattle
1140,15,858,5.0,997938703,858.0,Sleepless in Seattle
2730,17,858,5.0,1127469000,858.0,Sleepless in Seattle
3324,19,858,5.0,855191478,858.0,Sleepless in Seattle
3564,20,858,2.0,1238729822,858.0,Sleepless in Seattle
3678,21,858,4.0,853850728,858.0,Sleepless in Seattle
3823,22,858,4.0,1131662354,858.0,Sleepless in Seattle
4093,23,858,5.0,1148670263,858.0,Sleepless in Seattle


In [54]:
predicted_movies_by_name

defaultdict(list,
            {1: ['Sleepless in Seattle',
              'The Thomas Crown Affair',
              'Shriek If You Know What I Did Last Friday the Thirteenth',
              'Galaxy Quest',
              'Terminator 3: Rise of the Machines',
              'The Million Dollar Hotel',
              'Nell',
              'My Darling Clementine',
              'The 39 Steps',
              'Birdman of Alcatraz'],
             2: ['Lonely Hearts',
              'Sleepless in Seattle',
              'Straw Dogs',
              'The Thomas Crown Affair',
              'Galaxy Quest',
              'Hard Target',
              'Point Break',
              'The Million Dollar Hotel',
              'License to Wed',
              'Nell'],
             3: ['Sleepless in Seattle',
              'Lonely Hearts',
              'The Thomas Crown Affair',
              'Galaxy Quest',
              'Hard Target',
              'While You Were Sleeping',
              'Shriek If You Know 

In [77]:

def print_user_prediction(userId, predictions_dict):

    users_viewed_movies = ratings_with_movie_names[ratings_with_movie_names['userId'] == userId][['rating', 'original_title']]
    print(f'User {userId} has viewed the following movies:\n')

    for row in users_viewed_movies.itertuples():
        rating = row[1]
        original_title = row[2]
        print(f'\t{original_title}, Rating: {rating}')
        
    print(f'\nThe following movies are recommended for User {userId}\n')
    recommended_movies = [get_movie_name(mov_id[0]) for mov_id in predictions_dict[userId]]
    
    for movie in recommended_movies:
        print(f'\t{movie}')

In [78]:
print_user_prediction(9999999, top_n)

User 9999999 has viewed the following movies:

	There Will Be Blood, Rating: 5.0
	Boogie Nights, Rating: 4.5
	Rosemary's Baby, Rating: 4.0
	A Night at the Roxbury, Rating: 2.5

The following movies are recommended for User 9999999

	The Million Dollar Hotel
	Dawn of the Dead
	Hard Target
	Nell
	Galaxy Quest
	Birdman of Alcatraz
	The 39 Steps
	Bonnie and Clyde
	Sleepless in Seattle
	Cool Hand Luke
	While You Were Sleeping
	Dr. Jekyll and Mr. Hyde
	K-19: The Widowmaker
	Far from the Madding Crowd
	Frankenstein
	Lonely Hearts
	Shriek If You Know What I Did Last Friday the Thirteenth
	Local Color
	Dog Day Afternoon
	In Time
	Straw Dogs
	Rumble Fish
	Gremlins 2: The New Batch
	Loose Screws
	Confession of a Child of the Century
	End of the World
	Boat
	My Darling Clementine
	The Good Thief
	This Is 40
	Harry Potter and the Philosopher's Stone
	Within the Woods
	The Wanderers
	Grizzly Man
	The Bachelor
	Stranger Than Fiction
	Murder She Said
	Dr. Jekyll and Mr. Hyde
	Meet the Robinsons
	Space

#### Hybrid Rec System

Given that I have a user and a group of suggested films -> Segment them by genre. 

In [90]:
[print(v) for k, v in top_n.items()]

[(1252, 4.159909059209089), (858, 4.100142188818884), (913, 4.0891441016630985), (2959, 4.082554648043566), (318, 4.069255217846467), (2064, 4.067936582991246), (926, 3.983016975294147), (608, 3.980788799258147), (6016, 3.969883171548897), (1089, 3.939825952737934), (912, 3.934656526772218), (1968, 3.9212465804567342), (4226, 3.917098116746404), (3035, 3.914734093998166), (232, 3.9137084490416916), (1945, 3.904203454483093), (2019, 3.900074871950383), (58559, 3.8821707472242393), (2300, 3.8756065073057675), (750, 3.8745668207759207), (1267, 3.8683199671919946), (2324, 3.8662008253097158), (46976, 3.86384580637771), (4011, 3.857857108967795), (968, 3.8476145064796476), (1250, 3.8456101568089336), (7090, 3.8331515603162902), (3088, 3.8296188655933765), (3022, 3.8152715201453997), (296, 3.8100653864283505), (903, 3.809919521407391), (2762, 3.8095304137093953), (3683, 3.8055275559609845), (162, 3.7942895224971505), (44191, 3.7831120921722157), (2289, 3.782610548190199), (1411, 3.7738679202

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

Top n is a dictionary of Tuples. The Tuples are (movie-id, predicted-rating)

In [92]:
movie_dat_df = movies_df

In [93]:
movie_dat_df.head()

,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_title,overview,popularity,poster_path,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,-9223372036854775808,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,-9223372036854775808,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,-9223372036854775808,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,-9223372036854775808,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
8,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,-9223372036854775808,Sudden Death,International action superstar Jean Claude Van...,5.23158,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0


In [98]:
user_movie_recs_ids = [tup_value[0] for tup_value in top_n[9999999]]

In [108]:
key_df = pd.DataFrame(user_movie_recs_ids, columns=['id'])

In [120]:
def get_rec_movies_df(movie_rec_tuple_list, movie_df, key_column_name='id'):
    key_df = pd.DataFrame(movie_rec_tuple_list, columns=[key_column_name, 'estimated_rating'])
    #key_df = pd.DataFrame(movie_ids_list, columns=[key_column_name])
    return movie_df.merge(key_df, on='id')
    

In [112]:
movie_dat_df.merge(key_df, on='id').shape

(100, 21)

In [126]:
get_rec_movies_df(top_n[9999999], movie_dat_df)

,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_title,overview,popularity,poster_path,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,estimated_rating
0,NaN,31000000,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,1945,-9223372036854775808,Nell,"In a remote woodland cabin, a small town docto...",5.93096,/4XLF6QkUqMDL5SLDEw1nrpSMk8N.jpg,...,106683817.0,112.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Her heart. Her soul. Her language are a myster...,Nell,False,6.1,128.0,4.539637
1,NaN,17000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,2064,-9223372036854775808,While You Were Sleeping,A love story built on a misunderstanding. A tr...,15.7836,/9StV7DX4ZtRZGJs9VcVEPvxIKL6.jpg,...,182057016.0,103.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A story about love at second sight.,While You Were Sleeping,False,6.5,340.0,4.389025
2,NaN,11500000,"[{'id': 18, 'name': 'Drama'}]",NaN,2788,-9223372036854775808,Reality Bites,A small circle of friends suffering from post-...,6.51459,/gbi7dmxPNVqJQpKnHOpZ9dmeWAO.jpg,...,20079850.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A comedy about love in the '90s,Reality Bites,False,6.5,180.0,4.212701
3,"{'id': 85861, 'name': 'Beverly Hills Cop Colle...",50000000,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",NaN,306,-9223372036854775808,Beverly Hills Cop III,Back in sunny southern California and on the t...,11.7878,/tw9gAhqQcBFX0X0XfVbWqUsmzoU.jpg,...,119208989.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Beverly Hills Cop III,False,5.5,445.0,4.168147
4,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://www.mgm.com/#/our-titles/215/Blown-Away,178,-9223372036854775808,Blown Away,Blown Away tells the story of Jimmy Dove who w...,13.1551,/elDZEW6wkGHTAwIUDrNMARNgxPY.jpg,...,30156002.0,121.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,5. 4. 3. 2. 1......Time's Up.,Blown Away,False,6.0,126.0,4.080522
5,"{'id': 412680, 'name': 'Hard Target Collection...",18000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,2019,-9223372036854775808,Hard Target,"When a woman's father goes missing, she enlist...",7.83435,/6WEu60V7EzncuFJSVmGJzhFvs4I.jpg,...,74189677.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Don't hunt what you can't kill.,Hard Target,False,6.1,237.0,4.547498
6,NaN,21000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,858,-9223372036854775808,Sleepless in Seattle,A young boy who tries to set his dad up on a d...,10.2349,/afkYP15OeUOD0tFEmj6VvejuOcz.jpg,...,227799884.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"What if someone you never met, someone you nev...",Sleepless in Seattle,False,6.5,630.0,4.452835
7,NaN,22000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",NaN,581,-9223372036854775808,Dances with Wolves,"Wounded Civil War soldier, John Dunbar tries t...",11.6543,/hpmclspug1I8EwKSWhL7pWWltA.jpg,...,424208848.0,181.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Inside everyone is a frontier waiting to be di...,Dances with Wolves,False,7.6,1084.0,4.134624
8,NaN,80000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://www.warnerbros.com/archive/spacejam/mov...,2300,-9223372036854775808,Space Jam,In a desperate attempt to win a basketball mat...,11.0543,/bJhVLribUKCrKv1h1WFqv4QmRWM.jpg,...,250200000.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Get ready to jam.,Space Jam,False,6.5,1335.0,4.230588
9,NaN,1020000,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",NaN,1939,-9223372036854775808,Laura,A police detective falls in love with the woma...,9.88452,/fqFY81twXBjTAmDGw1fUooAvXf9.jpg,...,2000000.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The story of a love that became the most fearf...,Laura,False,7.6,173.0,4.223396


In [116]:
pd.DataFrame(top_n[9999999], columns=['id', 'estimated_rating'])

,id,estimated_rating
0,318,4.736636
1,923,4.640634
2,2019,4.547498
3,1945,4.539637
4,926,4.508373
5,898,4.498933
6,260,4.492402
7,475,4.456741
8,858,4.452835
9,903,4.451257
